In [0]:
-- Mathematical solution, not mine.
-- Credits to theMachine0094 - https://www.reddit.com/r/adventofcode/comments/1pbzqcx/comment/nrzxxq1 (here I've coded only his part1)
--     and to manjaroman2    - https://github.com/manjaroman2/adventofcode2025/blob/c670a73c68218acca9044efb8eb6b35b610d13d8/dec2/dec2.py (here I've coded only his part2)
-- both parts computes more or less in the same time on DBSQL, seems the rust option from theMachine0094 is the fastest


from read_files('/Volumes/waldar/fabien/aoc2025/aoc2025_02_input.txt', format => 'text', wholeText => true) as t (value)
--from values('11-22,95-115,998-1012,1188511880-1188511890,222220-222224,1698522-1698528,446443-446449,38593856-38593862,565653-565659,824824821-824824827,2121212118-2121212124') as t (value)
|>    select explode(split(value, ','))     as ranges
|>    select split_part(ranges, '-', 1)     as strt_range
           , split_part(ranges, '-', 2)     as last_range
|>    select length(strt_range)             as d_low
           , length(last_range)             as d_upp
           , d_upp div 2                    as period
           , power(10, period)::bigint + 1l as pattern
           , strt_range::bigint             as lower
           , last_range::bigint             as upper
           , greatest( (lower div pattern)::bigint + case lower % pattern when 0 then 0l else 1l end, power(10, period - 1)::bigint ) as dmin
           , least   ( (upper div pattern)::bigint                                                  , power(10, period)::bigint - 1l) as dmax
           , case when dmin <= dmax then aggregate(sequence(dmin, dmax), 0l, (acc, x) -> acc + x * pattern) else 0 end as pt1
           , aggregate( sequence(d_low, d_upp)
                      , map()::map<int,array<int>>
                      , (acc, x) -> map_concat(acc, map(x, filter(sequence(1, (x div 2)::int), y -> try_mod(x, y) = 0)))
                      ) as divisors
|> aggregate sum(pt1) as part1
           , sum(aggregate( sequence(d_low, d_upp)
                          , array()::array<bigint>
                          , (acc, d) -> acc || aggregate( transform(divisors[d], k -> struct(k, floor( (1 - power(10, k * (d div k))) / (1 - power(10, k)))::bigint as q))
                                                        , array()::array<bigint>
                                                        , (azz, x) -> azz || filter( transform( sequence( greatest( power(10, (x.k - 1)), lower div x.q)::bigint
                                                                                                        , least  ( power(10, x.k), ((upper div x.q) + 1))::bigint - 1l)
                                                                                              , c -> c * x.q )
                                                                                   , f -> f between lower and upper and f > 10)
                                                        , azz -> array_distinct(azz)
                                                        )
                          , acc -> aggregate(acc, 0l, (add, z) -> add + z)
                          )) as part2;


-- My own naive solution, testing every numbers between the range against regexp, but takes 1.3ish second to compute
-- I wrote this one using pipe syntax because I liked it's readability, chaining multiple selects.

from read_files('/Volumes/waldar/fabien/aoc2025/aoc2025_02_input.txt', format => 'text', wholeText => true) as t (value)
--from values('11-22,95-115,998-1012,1188511880-1188511890,222220-222224,1698522-1698528,446443-446449,38593856-38593862,565653-565659,824824821-824824827,2121212118-2121212124') as t (value)
|>    select explode(split(value, ',')) as ranges
|>    select split_part(ranges, '-', 1)::bigint as strt_range
           , split_part(ranges, '-', 2)::bigint as last_range
           , sequence(strt_range, last_range)   as theseq
|>    select filter(theseq, f -> regexp_like(f::string, r'^(\d+)\1$' )) as pt1
           , filter(theseq, f -> regexp_like(f::string, r'^(\d+)\1+$')) as pt2
|>    select explode(arrays_zip(pt1, pt2)) as expld
|> aggregate sum(expld.pt1) as part1
           , sum(expld.pt2) as part2;
